# 03 Market Simulator

In [1]:
!conda env list

# conda environments:
#
base                     /home/ubuntu/anaconda3
alert                    /home/ubuntu/anaconda3/envs/alert
aws_neuron_mxnet_p36     /home/ubuntu/anaconda3/envs/aws_neuron_mxnet_p36
aws_neuron_tensorflow_p36     /home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36
chainer_p27              /home/ubuntu/anaconda3/envs/chainer_p27
chainer_p36              /home/ubuntu/anaconda3/envs/chainer_p36
ml                    *  /home/ubuntu/anaconda3/envs/ml
mxnet_p27                /home/ubuntu/anaconda3/envs/mxnet_p27
mxnet_p36                /home/ubuntu/anaconda3/envs/mxnet_p36
python2                  /home/ubuntu/anaconda3/envs/python2
python3                  /home/ubuntu/anaconda3/envs/python3
pytorch_p27              /home/ubuntu/anaconda3/envs/pytorch_p27
pytorch_p36              /home/ubuntu/anaconda3/envs/pytorch_p36
tensorflow2_p27          /home/ubuntu/anaconda3/envs/tensorflow2_p27
tensorflow2_p36          /home/ubuntu/anaconda3/envs/tensorflow2_p36
tensorflo

In [2]:
import sys
from pathlib import Path
# add project folder to sys path
sys.path.append(str(Path.cwd().parent))

In [3]:
# import libraries
import datetime as dt
import numpy as np
import pandas as pd
from util import get_data, plot_data

In [4]:
CURRENT_DIR = Path.cwd()
PROJECT_DIR = Path.cwd().parent
ORDER_DIR = PROJECT_DIR / "orders"

In [9]:
# check orders file
orders_file = str(ORDER_DIR/"orders.csv")
df_orders = pd.read_csv(orders_file, index_col='Date', parse_dates=True, na_values=['nan'])
df_orders

,Symbol,Order,Shares
Date,,,
2011-01-10,AAPL,BUY,1500
2011-01-13,AAPL,SELL,1500
2011-01-13,IBM,BUY,4000
2011-01-26,GOOG,BUY,1000
2011-02-02,XOM,SELL,4000
2011-02-10,XOM,BUY,4000
2011-03-03,GOOG,SELL,1000
2011-03-03,IBM,SELL,2200
2011-06-03,IBM,SELL,3300


In [25]:
def compute_portvals(orders_file="./orders/orders.csv", start_val=1000000, commission=9.95, impact=0.005):
    # NOTE: orders_file may be a string, or it may be a file object. 

    # 1. Import Order dataframe
    df_orders = pd.read_csv(orders_file, index_col='Date', parse_dates=True, na_values=['nan'])
    # print("df_orders")
    # print(df_orders.head())

    # Bonus: Don't Calculate 2011-6-15 order

    # 2. Sort order file by dates (ascending)
    df_orders = df_orders.sort_index(ascending=1)
    # print("df_orders")
    # print(df_orders)
    # print(df_orders.shape)

    # 3. Get symbols for the portfolio
    symbols = df_orders["Symbol"].unique().tolist()
    # print("symbols")
    # print(symbols)
    # print(type(symbols))

    # 4. Get date range
    start_date = min(df_orders.index)
    end_date = max(df_orders.index)
    # print("start_date", start_date)
    # print(type(start_date))
    # print("end_date", end_date)
    # print(type(end_date))

    # 5. Get df_prices using adjusted closing price, add cash column at last (all 1.0)
    df_prices = get_data(symbols, pd.date_range(start_date, end_date))
    # sort by date
    df_prices = df_prices.sort_index(ascending=1)
    # print("df_prices")
    # print(df_prices.head())
    df_prices_SPY = df_prices['SPY']
    # drop SPY column
    df_prices = df_prices.drop(['SPY'], axis=1)
    # add cash column
    df_prices['CASH'] = 1.0
    # add index name
    df_prices.index.name = 'DATE'
    # print(df_prices)

    # 6. Get df_trades using df_orders and df_prices
    df_trades = pd.DataFrame(0., columns=df_prices.columns, index=df_prices.index)
    # print("df_trades")
    # print(df_trades)
    for index, row in df_orders.iterrows():
        # print(df_prices.loc[[index], [row['Symbol']]])

        # if SELL symbol volume should be x(-1), otherwise x1
        sign = -1.0 if row['Order'] == 'SELL' else 1.0
        transaction_volume = sign * int(row['Shares'])
        df_trades.loc[[index], [row['Symbol']]] += transaction_volume
        # CASH is changing in the opposite direction, x (-1), could be multiple trades per day so use "+=" to update
        transaction_price = df_prices.loc[[index], [row['Symbol']]].values[0]
        df_trades.loc[[index], ['CASH']] += transaction_price * (-1) * transaction_volume

        ## Part 2 Transaction Costs
        # 2.1 Deduct transaction commission from CASH account for each trade
        df_trades.loc[[index], ['CASH']] -= commission

        # 2.2 Deduct market impact from CASH account for each trade
        market_impact = int(row['Shares']) * transaction_price * impact
        df_trades.loc[[index], ['CASH']] -= market_impact

    # print("df_trades")
    # print(df_trades)

    # 7. Get df_holdings
    df_holdings = pd.DataFrame(0., columns=df_trades.columns, index=df_trades.index)
    # print("df_holdings")
    # print(df_holdings)

    # initialize first row of df_holdings
    df_holdings.iloc[0] = df_trades.iloc[0]
    df_holdings.iloc[0]['CASH'] += start_val
    # print(df_holdings.iloc[0])

    for i in range(1, len(df_holdings)):
        df_holdings.iloc[i] = df_holdings.iloc[i - 1] + df_trades.iloc[i]
    # print("df_holdings")
    # print(df_holdings)

    # 8. Get df_values SUM(symbol_volume * symbol price) using df_holdings & df_prices
    df_values = pd.DataFrame(0., columns=df_holdings.columns, index=df_holdings.index)
    # print("df_values")
    # print(df_values)

    # Use element-wise multiplication
    df_values = df_prices * df_holdings
    # print(df_values)

    # 9. Get portvals by using row sum of df_values (axis=1)
    portvals = df_values.sum(axis=1)
    # print("portvals")
    # print(portvals)

    return portvals

In [26]:
def test_code():
    # Define input parameters
    of = "../orders/orders-02.csv"
    sv = 1000000

    # Process orders
    portvals = compute_portvals(orders_file=of, start_val=sv)
    if isinstance(portvals, pd.DataFrame):
        portvals = portvals[portvals.columns[0]]  # just get the first column
    else:
        "warning, code did not return a DataFrame"

    # Get portfolio stats
    # Here we just fake the data. you should use your code from previous assignments.
    start_date = dt.datetime(2008, 1, 1)
    end_date = dt.datetime(2008, 6, 1)
    cum_ret, avg_daily_ret, std_daily_ret, sharpe_ratio = [
        0.2, 0.01, 0.02, 1.5]
    cum_ret_SPY, avg_daily_ret_SPY, std_daily_ret_SPY, sharpe_ratio_SPY = [
        0.2, 0.01, 0.02, 1.5]

    # Compare portfolio against $SPX
    print("Date Range: {} to {}".format(start_date, end_date))
    print()
    print("Sharpe Ratio of Fund: {}".format(sharpe_ratio))
    print("Sharpe Ratio of SPY : {}".format(sharpe_ratio_SPY))
    print()
    print("Cumulative Return of Fund: {}".format(cum_ret))
    print("Cumulative Return of SPY : {}".format(cum_ret_SPY))
    print()
    print("Standard Deviation of Fund: {}".format(std_daily_ret))
    print("Standard Deviation of SPY : {}".format(std_daily_ret_SPY))
    print()
    print("Average Daily Return of Fund: {}".format(avg_daily_ret))
    print("Average Daily Return of SPY : {}".format(avg_daily_ret_SPY))
    print()
    print("Final Portfolio Value: {}".format(portvals[-1]))

In [27]:
test_code()

Date Range: 2008-01-01 00:00:00 to 2008-06-01 00:00:00

Sharpe Ratio of Fund: 1.5
Sharpe Ratio of SPY : 1.5

Cumulative Return of Fund: 0.2
Cumulative Return of SPY : 0.2

Standard Deviation of Fund: 0.02
Standard Deviation of SPY : 0.02

Average Daily Return of Fund: 0.01
Average Daily Return of SPY : 0.01

Final Portfolio Value: 1064130.0237500002
